In [1]:
import geopandas as gpd
import rasterio
import pandas as pd
import numpy as np
import rasterio
from skimage.feature import graycomatrix, graycoprops
from sklearn.cluster import KMeans
import warnings
warnings.filterwarnings("ignore") 
warnings.filterwarnings("ignore", category=UserWarning)

### Load sample points and sar data


In [2]:
## palisades
shp_path = "../../sample_collection/palisades_random_sample.shp"
# shp_path = "../../sample_collection/dump/refined_sample.shp"

gdf = gpd.read_file(shp_path)

# input_sar_image_path  = "../../SAR_Data_processing/8_1_project/S1A_IW_GRDH_1SDV_palisades_32611_bilinear_10.tif"
glcm_raster_path="input/eaton_glcm_ready.tif"
input_sar_image_path  = "input/palisades_s1_ready.tif"

thermal_image_path="input/palisades_thermal_ready.tif"
dnbr_image_path="input/palisades_dnbr_ready.tif"
add_bands=False
add_indices=True
add_glcm=False
thermal_band=False
add_dnbr=False

final_columns_names=["RBD_VV", "RBD_VH", "RBR_VV", "RBR_VH","ΔRVI",'Δvv_vh_ratio',"ΔRDFI","RCBI"]
output_sar_image_path="output/feature_image/palisades_sar_indices_d.tif"
output_file_path="output/sample_feature/palisades_sar_indices_d.csv"


# final_columns_names=["dTRAD","dNBR","RBD_VV", "RBD_VH", "RBR_VV", "RBR_VH","ΔRVI",'Δvv_vh_ratio',"ΔRDFI","RCBI"]
# output_sar_image_path="output/feature_image/palisades_sar_optical_indices.tif"
# output_file_path="output/sample_feature/palisades_sar_optical_indices.csv"


In [6]:
##test data
input_sar_image_path="../../SAR_Data_processing/8_0_subset/exported_from_eaton/palisades_S1A_IW_GRDH_1SDV_2024_12_16_2024_01_21_TC_stack_32611_bilinear_10.tif"

final_columns_names=["RBD_VV", "RBD_VH", "RBR_VV", "RBR_VH","ΔRVI",'Δvv_vh_ratio',"ΔRDFI","RCBI"]
output_sar_image_path="test/sar_indices.tif"

In [3]:
# # Load training points
# shp_path = "../../sample_collection/eaton_random_sample.shp"
# gdf = gpd.read_file(shp_path)

# input_sar_image_path  = "input/eaton_s1_ready.tif"
# glcm_raster_path="input/eaton_glcm_ready.tif"
# thermal_image_path="input/eaton_thermal_ready.tif"
# dnbr_image_path="input/eaton_dnbr_ready.tif"
# #following are the input for the model selection and training

# # final_columns_names=["RBD_VV", "RBD_VH", "RBR_VV", "RBR_VH","ΔRVI",'Δvv_vh_ratio',"ΔRDFI","RCBI"]
# # output_file_path="output/sample_feature/eaton_sar_indices.csv"
# # output_sar_image_path="output/feature_image/eaton_sar_indices.tif"

# final_columns_names=["dTRAD","dNBR","RBD_VV", "RBD_VH", "RBR_VV", "RBR_VH","ΔRVI",'Δvv_vh_ratio',"ΔRDFI","RCBI"]
# output_file_path="output/sample_feature/eaton_sar_optical_indices.csv"
# output_sar_image_path="output/feature_image/eaton_sar_optical_indices.tif"

# # final_columns_names=["vh_post","vv_post","vh_pre","vv_pre"]
# # "dTRAD","dNBR","dod_RBR_VH",dod_RBR_VV
# # RCBI: https://www.sciencedirect.com/science/article/abs/pii/S0924271619302242
# # final_columns_names=['t1','t2','t3','t4','t5','t6','t7','t8','t9','t10','t11','t12','t13','t14','t15','t16','t17','t18','t19','t20']


# add_bands=False
# add_indices=True
# add_glcm=False
# thermal_band=True
# add_dnbr=True

In [7]:
"""
Log10 of negative value is 0, thus applicanle in calculating the differnce
"""
from scipy.ndimage import generic_filter

# Open input raster
with rasterio.open(input_sar_image_path) as src:
    profile = src.profile  # Get metadata
    bands = src.read()  # Read all bands

profile.update(count=len(final_columns_names))
print(len(final_columns_names),"len(final_columns_names)")
epsilon = 1e-6
band_index=0
with rasterio.open(output_sar_image_path, "w", **profile) as dst:
    if add_bands:
        for i in range(len(bands)):
            dst.write(bands[i], i + 1)
            dst.set_band_description(i + 1, final_columns_names[i])  # Assign names
            band_index=i+1

    
    if thermal_band:
        with rasterio.open(thermal_image_path) as TRAD_dataset:
            
            # profile = src.profile  # Get metadata
            trad_band = TRAD_dataset.read()
            band_index=band_index+1
            dst.write(trad_band[0], band_index)
            dst.set_band_description(band_index, "dTRAD")
    
    if add_dnbr:
        with rasterio.open(dnbr_image_path) as dnbr_dataset:
            # profile = src.profile  # Get metadata
            dnbr_band = dnbr_dataset.read()
            band_index=band_index+1
            dst.write(dnbr_band[0], band_index)
            dst.set_band_description(band_index, "dNBR")
    
    if add_indices:
        # vh_post,vv_post,vh_pre, vv_pre=bands[0],bands[1],bands[2],bands[3]
        vh_pre,vv_pre,vh_post, vv_post=bands[0],bands[1],bands[2],bands[3]
        vv_post = np.nan_to_num(vv_post, nan=0)
        vh_post = np.nan_to_num(vh_post, nan=0)
        vv_pre = np.nan_to_num(vv_pre, nan=0)
        vh_pre = np.nan_to_num(vh_pre, nan=0)

        vv_post = np.where(vv_post <= 0, epsilon, vv_post)
        vh_post = np.where(vh_post <= 0, epsilon, vh_post)
        vv_pre = np.where(vv_pre <= 0, epsilon, vv_pre)
        vh_pre = np.where(vh_pre <= 0, epsilon, vh_pre)

        # vh_post,vv_post,vh_pre, vv_pre=(bands[2]+bands[6])/2,(bands[3]+bands[7])/2,(bands[0]+bands[4])/2,(bands[1]+bands[5])/2

        # print(np.min(vh_post),np.max(vh_post))
        band_index=band_index+1
        RBD_VV_band = vv_post - vv_pre
        # X_min,X_max = np.min(RBD_VV_band),np.max(RBD_VV_band)
        # RBD_VV_band = (RBD_VV_band - X_min) / (X_max - X_min)
        dst.write(RBD_VV_band, band_index)
        dst.set_band_description(band_index, "RBD_VV")


        band_index=band_index+1
        RBD_VH_band = vh_post - vh_pre
        # X_min,X_max = np.min(RBD_VH_band),np.max(RBD_VH_band)
        # RBD_VH_band = (RBD_VH_band - X_min) / (X_max - X_min)
        dst.write(RBD_VH_band, band_index)
        dst.set_band_description(band_index, "RBD_VH")

        band_index=band_index+1
        # RBR_VV_band = vv_post/ (vv_pre + epsilon)
        RBR_VV_band = (np.log10(vv_post))-(np.log10(vv_pre)) #logarithmic ratio
        # RBR_VV_band=np.log(RBR_VV_band + epsilon) 
        # X_min,X_max = np.min(RBR_VV_band),np.max(RBR_VV_band)
        # RBR_VV_band = (RBR_VV_band - X_min) / (X_max - X_min)
        dst.write(RBR_VV_band, band_index)
        dst.set_band_description(band_index, "RBR_VV")
        
        # band_index=band_index+1
        # # pre_std=np.std(vv_pre) #global
        # window_size = 7
        # # Apply local standard deviation filter
        # pre_std_local = generic_filter(vv_pre, np.std, size=window_size)
        # RBR_VV_band=(np.log10(vv_post))-(np.log10(vv_pre)) #logarithmic ratio
        # dod_RBR_VV= np.abs(RBR_VV_band)/pre_std_local
        # dst.write(dod_RBR_VV, band_index)
        # dst.set_band_description(band_index, "dod_RBR_VV")


        band_index=band_index+1
        # RBR_VH_band = vh_post / (vh_pre+epsilon)
        RBR_VH_band = (np.log10(vh_post)) - (np.log10(vh_pre))
        # RBR_VH_band=np.log(RBR_VH_band + epsilon) 
        # X_min,X_max = np.min(RBR_VH_band),np.max(RBR_VH_band)
        # RBR_VH_band = (RBR_VH_band - X_min) / (X_max - X_min)
        dst.write(RBR_VH_band, band_index)
        dst.set_band_description(band_index, "RBR_VH")

        # band_index=band_index+1
        # # pre_std=np.std(vh_pre)
        # window_size = 7
        # # Apply local standard deviation filter
        # pre_std_local = generic_filter(vv_pre, np.std, size=window_size)
        # RBR_VH_band=(np.log10(vh_post))-(np.log10(vh_pre))
        # dod_RBR_VH= np.abs(RBR_VH_band)/pre_std_local
        # dst.write(dod_RBR_VH, band_index)
        # dst.set_band_description(band_index, "dod_RBR_VH")

        band_index=band_index+1
        # RVI_post = 4* vh_post/(vv_post+vh_post+epsilon) #4* vh/(vv+vh)
        # RVI_pre = 4* vh_pre/(vh_pre+vv_pre+epsilon) #4* vh/(vv+vh)
        RVI_post = (10*np.log10((4* vh_post) ))-(10*np.log10(vv_post+vh_post)) #4* vh/(vv+vh)
        RVI_pre = (10*np.log10((4* vh_pre)))-(10*np.log10(vh_pre+vv_pre)) #4* vh/(vv+vh)
        delta_RVI = RVI_post  - RVI_pre
        # delta_RVI=10*np.log10(delta_RVI+epsilon)
        # delta_RVI = np.log(RVI_post + epsilon) - np.log(RVI_pre + epsilon)
        # X_min, X_max = np.min(delta_RVI), np.max(delta_RVI)
        # delta_RVI = (delta_RVI - X_min) / (X_max - X_min)
        dst.write(delta_RVI, band_index)
        dst.set_band_description(band_index, "ΔRVI")

        band_index=band_index+1
        # vv_vh_ratio_post = vv_post/(vh_post+epsilon) 
        # vv_vh_ratio_pre = vv_pre/(vh_pre+epsilon)
        vv_vh_ratio_post = (10*np.log10(vv_post))-(10*np.log10(vh_post))
        vv_vh_ratio_pre = (10*np.log10(vv_pre))-(10*np.log10(vh_pre))
        delta_vv_vh_ratio=vv_vh_ratio_post-vv_vh_ratio_pre
        # delta_vv_vh_ratio=10*np.log10(delta_vv_vh_ratio+epsilon)
        # X_min, X_max = np.min(delta_vv_vh_ratio), np.max(delta_vv_vh_ratio)
        # delta_vv_vh_ratio = (delta_vv_vh_ratio - X_min) / (X_max - X_min)
        # delta_vv_vh_ratio=np.log(vv_vh_ratio_post+epsilon)-np.log(vv_vh_ratio_pre+epsilon)
        dst.write(delta_vv_vh_ratio, band_index)
        dst.set_band_description(band_index, "Δ_vv_vh_ratio")

        band_index=band_index+1
        RFDI_pre = (vv_pre - vh_pre) / (vv_pre + vh_pre)
        RFDI_post = (vv_post - vh_post) / (vv_post + vh_post)
        delta_RFDI = RFDI_post - RFDI_pre
        # X_min, X_max = np.min(delta_RFDI), np.max(delta_RFDI)
        # delta_RFDI = (delta_RFDI - X_min) / (X_max - X_min)
        dst.write(delta_RFDI, band_index)
        dst.set_band_description(band_index, "ΔRDFI")

        band_index=band_index+1
        #Radar convolution burn index
        RCBI=(vv_post-vv_pre)+(vh_post-vh_pre)
        dst.write(RCBI, band_index)
        dst.set_band_description(band_index, "RCBI")
    
    if add_glcm:
        # Open input raster
        with rasterio.open(glcm_raster_path) as glcm_dst:
            # profile = src.profile  # Get metadata
            glcm_bands = glcm_dst.read()  # Read all bands
            # print(len(glcm_bands))

        # glcm_bands_counts=len(glcm_bands)/2

        band_index=band_index+1
        with rasterio.open(glcm_raster_path) as glcm_dataset:
            glcm_bands = glcm_dataset.read()  # Read all bands
            # for i in range(16,20):
            #     difference=glcm_bands[i]-glcm_bands[i+20]
            #     dst.write(difference, band_index)
            #     band_index=band_index+1
            #     break
            for i in range(int(len(glcm_bands)/2)):
                corresponding_index,index,  = i, int(len(bands)/2)+i
                difference=glcm_bands[corresponding_index]-glcm_bands[index]
                X_min, X_max = np.min(difference), np.max(difference)
                print(X_max, X_min)
                difference = (difference - X_min) / (X_max - X_min)
                dst.write(difference+epsilon, band_index)
                band_index=band_index+1
    

8 len(final_columns_names)


In [5]:
# dataset=rasterio.open()
with rasterio.open(output_sar_image_path) as dataset:
    print(dataset.count)
    # Get coordinates of training points
    coords = [(x, y) for x, y in zip(gdf.geometry.x, gdf.geometry.y)]

    # Extract values from all bands
    values = [v for v in dataset.sample(coords)]

    # Convert to DataFrame with proper band names
    df = pd.DataFrame(values, columns=final_columns_names)

    # Merge with training sample points
    gdf = gdf.reset_index(drop=True)
    final_df = pd.concat([gdf, df], axis=1)

    # Save as CSV for ML training
    final_df.to_csv(output_file_path, index=False)

8


In [6]:
# final_df.head(10)

In [7]:
df.agg(['min', 'max'])

,RBD_VV,RBD_VH,RBR_VV,RBR_VH,ΔRVI,Δvv_vh_ratio,ΔRDFI,RCBI
min,-2.236699,-0.380620,-1.549543,-1.279708,-7.286319,-12.322313,-0.550073,-2.444001
max,0.981616,0.148307,0.594986,0.701436,11.390145,9.175694,0.774088,1.129923


In [8]:
# 	dTRAD	dNBR	RBD_VV	RBD_VH	RBR_VV	dod_RBR_VV	RBR_VH	dod_RBR_VH	ΔRVI	Δvv_vh_ratio	ΔRDFI	RCBI
# min	0.057610	-0.248237	-1.953554	-0.422484	-0.756728	0.000416	-1.267929	0.000264	-14.016992	-9.988633	-0.875591	-2.064492
# max	0.958609	0.739910	5.741815	0.638357	1.805561	216.663483	1.513628	185.473175	7.720948	15.615835	1.214909	5.602402